In [1]:
from itertools import tee
from typing  import Tuple
from helpers import data

In [2]:
bp_seats = data(5)
bp_seats[:5]

['FBFBFFBLLR', 'FBBBFFBLLL', 'BFBFFFFRRR', 'BFBBBFBRLL', 'FBFBBFBLLL']

**Part 1:** Find the highest seat ID

In [3]:
def seat_row_column(bp_seat) -> Tuple[int, int]: 
    """Convert binary-partitioned seat (7 characters F/B), 
    (3 characters L/R) to a tuple (row, column).
    """
    row_bp = bp_seat[:7]
    col_bp = bp_seat[-3:]
    # Row 
    row = 0 
    row_max = 128
    for i, binary in enumerate(row_bp):
        row += row_max / 2**(i+1) * (binary == "B")
    # Column 
    col = 0 
    col_max = 8 
    for i, binary in enumerate(col_bp):
        col += col_max / 2**(i+1) * (binary == "R")
    return tuple(map(int, (row, col)))

def seat_id(bp_seat) -> int: 
    row, col = seat_row_column(bp_seat)
    return row * 8 + col

In [4]:
# Attempt 1
max(seat_id(bp_seat) for bp_seat in bp_seats)

938

`seat_row_column` is ugly. How do I make it better?

In [5]:
def binary_partitioned_to_value(bp: str, one_char: chr = "1") -> int: 
    """Decode binary-partitioned string where one_char means 1/yes."""
    val = 0
    for i, c in enumerate(bp): 
        # I think this is clean enough...
        val += 2**(len(bp) - (i+1)) * (c == one_char)
    return val

def binary_to_row_col(bp_seat) -> Tuple[int, int]:
    """
    Convert a binary-partitioned seat (written as 7 characters F/B 
    followed by 3 characters L/R) to a tuple (row, column). 
    
    >>> binary_to_row_col("FBFBBFFRLR")
    (44, 5)
    >>> binary_to_row_col("BFFFBBFRRR")
    (70, 7)
    >>> binary_to_row_col("FFFBBBFRRR")
    (14, 7)
    >>> binary_to_row_col("BBFFBBFRLL")
    (102, 4)
    """
    binary_row = bp_seat[:7]
    binary_col = bp_seat[7:]
    return (binary_partitioned_to_value(binary_row, "B"), 
            binary_partitioned_to_value(binary_col, "R")) 

def seat_id(bp_seat) -> int: 
    row, col = binary_to_row_col(bp_seat)
    return row * 8 + col

In [6]:
# Attempt 3
max(seat_id(bp_seat) for bp_seat in bp_seats)

938

In [7]:
# Dangit Norvig 
ID = int # Type 
def seat_id(seat: str, table=str.maketrans("FBLR", "0101")) -> ID:
    return ID(seat.translate(table), base=2)

max(seat_id(seat) for seat in bp_seats)

938

**Part 2:** My seat is missing from the list. Other seats are missing too, but my seat will be the only one where the seats with ID +1 and -1 will exist. Find my seat ID. 

In [8]:
def pairwise(iterable): 
    """s -> (s0, s1), (s1, s2), (s2, s3), ..."""
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

In [9]:
# Attempt 1 
ids = sorted([seat_id(bp_seat) for bp_seat in bp_seats])
for curr_id, next_id in pairwise(ids):
    if curr_id + 1 == next_id - 1:
        print(curr_id + 1)
        break

696


In [10]:
# Attempt 2
ids = set(seat_id(bp_seat) for bp_seat in bp_seats)
for curr_id in ids:
    if curr_id + 1 not in ids and curr_id + 2 in ids:
        print(curr_id + 1)
        break

696


In [11]:
# Dammit Norvig 
# I didn't use information that only other missing ones are at the bottom or end 
ids = set(map(seat_id, bp_seats))
[missing] = set(range(min(ids), max(ids))) - ids 

missing

696